# TensorRT Optimization - Google Colab GPU Test

이 노트북을 Google Colab에서 실행하여 실제 GPU 성능을 테스트합니다.

**중요**: Runtime → Change runtime type → GPU 선택 필수!

In [ ]:
# GPU 확인
!nvidia-smi

In [ ]:
# 레포지토리 클론 또는 업데이트
import os

if not os.path.exists('/content/nvidia-devtech-portfolio'):
    !git clone https://github.com/midmost44/nvidia-devtech-portfolio.git /content/nvidia-devtech-portfolio
else:
    !cd /content/nvidia-devtech-portfolio && git pull

%cd /content/nvidia-devtech-portfolio/projects/01-tensorrt-optimization

In [ ]:
# 필요한 패키지 설치
!pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu118
!pip install -q tensorrt pycuda
!pip install -q onnx onnxruntime-gpu
!pip install -q coloredlogs pynvml matplotlib seaborn tabulate tqdm
!pip install -q onnx-simplifier

In [ ]:
# Python path 설정
import sys
sys.path.append('/content/nvidia-devtech-portfolio/projects/01-tensorrt-optimization/src')

# GPU 확인
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## Step 1: PyTorch to ONNX 변환

In [ ]:
# 디렉토리 생성
!mkdir -p models engines results plots calibration_images

In [ ]:
# ONNX 변환
!python src/convert_to_onnx.py \
    --model resnet50 \
    --output models/resnet50.onnx \
    --batch-size 1 \
    --dynamic-batch

## Step 2: Calibration 데이터 생성 (INT8용)

In [ ]:
# Calibration 이미지 생성
!python src/calibration.py \
    --output calibration_images \
    --num-images 100 \
    --size 224 224

## Step 3: TensorRT 엔진 빌드

In [ ]:
# FP32 엔진 빌드
!python src/convert_to_tensorrt.py \
    --onnx models/resnet50.onnx \
    --output engines/resnet50_fp32.trt \
    --precision fp32 \
    --max-batch-size 16

In [ ]:
# FP16 엔진 빌드
!python src/convert_to_tensorrt.py \
    --onnx models/resnet50.onnx \
    --output engines/resnet50_fp16.trt \
    --precision fp16 \
    --max-batch-size 16

In [ ]:
# INT8 엔진 빌드 (calibration 필요)
!python src/convert_to_tensorrt.py \
    --onnx models/resnet50.onnx \
    --output engines/resnet50_int8.trt \
    --precision int8 \
    --max-batch-size 16 \
    --calibration-data calibration_images

## Step 4: 성능 벤치마킹

In [ ]:
# 전체 벤치마크 실행
!python src/benchmark.py \
    --pytorch-model resnet50 \
    --trt-engines engines \
    --batch-sizes 1 4 8 16 \
    --iterations 100 \
    --warmup 10 \
    --output results/benchmark.json

## Step 5: 결과 시각화

In [ ]:
# 시각화 생성
!python src/visualize_results.py \
    --results results/benchmark.json \
    --output plots

In [ ]:
# 결과 이미지 표시
from IPython.display import Image, display
import os

plot_files = [
    'plots/latency_comparison.png',
    'plots/throughput_scaling.png',
    'plots/speedup_heatmap.png',
    'plots/memory_comparison.png'
]

for plot_file in plot_files:
    if os.path.exists(plot_file):
        print(f"\n{plot_file}:")
        display(Image(plot_file))

## Step 6: 성능 요약

In [ ]:
# 벤치마크 결과 분석
import json

with open('results/benchmark.json', 'r') as f:
    results = json.load(f)

print("="*60)
print("PERFORMANCE SUMMARY")
print("="*60)

# GPU 정보
print(f"\nGPU: {results['metadata'].get('gpu', 'Unknown')}")
print(f"Model: {results['metadata']['model']}")
print(f"Input Size: {results['metadata']['input_size']}")

# 최고 성능 찾기
best_speedup = 0
best_config = ""

for batch_key, batch_results in results['benchmarks'].items():
    batch_size = int(batch_key.split('_')[1])
    
    if 'pytorch' in batch_results and 'fp32' in batch_results['pytorch']:
        baseline = batch_results['pytorch']['fp32']['mean_latency_ms']
        
        print(f"\nBatch Size {batch_size}:")
        print("-"*40)
        print(f"PyTorch FP32: {baseline:.2f} ms (baseline)")
        
        for precision in ['fp32', 'fp16', 'int8']:
            key = f'tensorrt_{precision}'
            if key in batch_results and 'mean_latency_ms' in batch_results[key]:
                trt_latency = batch_results[key]['mean_latency_ms']
                speedup = baseline / trt_latency
                
                print(f"TensorRT {precision.upper()}: {trt_latency:.2f} ms ({speedup:.2f}x speedup)")
                
                if speedup > best_speedup:
                    best_speedup = speedup
                    best_config = f"TensorRT {precision.upper()} (Batch={batch_size})"

print("\n" + "="*60)
print(f"🚀 Best Performance: {best_speedup:.2f}x speedup with {best_config}")
print("="*60)

In [ ]:
# 결과 파일 다운로드 준비
from google.colab import files
import shutil

# 결과 압축
shutil.make_archive('tensorrt_results', 'zip', '.', 'results')
shutil.make_archive('tensorrt_plots', 'zip', '.', 'plots')

print("Results ready for download:")
print("1. tensorrt_results.zip - Benchmark JSON data")
print("2. tensorrt_plots.zip - Visualization plots")

# 다운로드
files.download('tensorrt_results.zip')
files.download('tensorrt_plots.zip')